In [2]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

from name_dataset import NameDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [3]:
# Parameters and DataLoaders
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_EPOCHS = 100

test_dataset = NameDataset(is_train_set=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=True)


train_dataset = NameDataset(is_train_set=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

N_COUNTRIES = len(train_dataset.get_countries())
print(N_COUNTRIES, "countries")
N_CHARS = 128  # ASCII

18 countries


In [4]:
# Some utility functions
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def create_variable(tensor):
    # Do cuda() before wrapping with variable
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

In [5]:
# pad sequences and sort the tensor
def pad_sequences(vectorized_seqs, seq_lengths, countries):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)

    # Sort tensors by their length
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]

    # Also sort the target (countries) in the same order
    target = countries2tensor(countries)
    if len(countries):
        target = target[perm_idx]

    # Return variables
    # DataParallel requires everything to be a Variable
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)

In [6]:
# Create necessary variables, lengths, and target
def make_variables(names, countries):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths, countries)


def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)


def countries2tensor(countries):
    country_ids = [train_dataset.get_country_id(
        country) for country in countries]
    return torch.LongTensor(country_ids)


In [7]:
class RNNClassifier(nn.Module):
    # Our model

    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, seq_lengths):
        # Note: we run this all at once (over the whole input sequence)
        # input shape: B x S (input size)
        # transpose to make S(sequence) x B (batch)
        input = input.t()
        batch_size = input.size(1)

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        # Embedding S x B -> S x B x I (embedding size)
        embedded = self.embedding(input)

        # Pack them up nicely
        gru_input = pack_padded_sequence(
            embedded, seq_lengths.data.cpu().numpy())

        # To compact weights again call flatten_parameters().
        self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)

        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden[-1])
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size)
        return create_variable(hidden)

In [11]:
# Train cycle
def train():
    total_loss = 0

    for i, (names, countries) in enumerate(train_loader, 1):
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)

        loss = criterion(output, target)
        total_loss += loss.data

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                time_since(start), epoch,  i *
                len(names), len(train_loader.dataset),
                100. * i * len(names) / len(train_loader.dataset),
                total_loss / i * len(names)))

    return total_loss

In [9]:
# Testing cycle
def test(name=None):
    # Predict for a given name
    if name:
        input, seq_lengths, target = make_variables([name], [])
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        country_id = pred.cpu().numpy()[0][0]
        print(name, "is", train_dataset.get_country(country_id))
        return

    print("evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)

    for names, countries in test_loader:
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))

In [12]:
if __name__ == '__main__':

    classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)

    if torch.cuda.is_available():
        classifier.cuda()

    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    print("Training for %d epochs..." % N_EPOCHS)
    for epoch in range(1, N_EPOCHS + 1):
        # Train cycle
        train()

        # Testing
        test()

        # Testing several samples
        test("Sung")
        test("Jungwoo")
        test("Soojin")
        test("Nako")

Training for 100 epochs...
 1s] Train Epoch: 1 [2560/13374 (19%)]	Loss: 609.25
 3s] Train Epoch: 1 [5120/13374 (38%)]	Loss: 536.08
 4s] Train Epoch: 1 [7680/13374 (57%)]	Loss: 497.25
 6s] Train Epoch: 1 [10240/13374 (77%)]	Loss: 466.69
 8s] Train Epoch: 1 [12800/13374 (96%)]	Loss: 442.16
evaluating trained model ...

Test set: Accuracy: 4261/6700 (64%)

Sung is Vietnamese
Jungwoo is Russian
Soojin is English
Nako is Russian
 11s] Train Epoch: 2 [2560/13374 (19%)]	Loss: 316.02
 13s] Train Epoch: 2 [5120/13374 (38%)]	Loss: 310.08
 14s] Train Epoch: 2 [7680/13374 (57%)]	Loss: 305.86
 16s] Train Epoch: 2 [10240/13374 (77%)]	Loss: 301.25
 18s] Train Epoch: 2 [12800/13374 (96%)]	Loss: 296.70
evaluating trained model ...

Test set: Accuracy: 4632/6700 (69%)

Sung is Chinese
Jungwoo is Russian
Soojin is Russian
Nako is Japanese
 22s] Train Epoch: 3 [2560/13374 (19%)]	Loss: 253.65
 24s] Train Epoch: 3 [5120/13374 (38%)]	Loss: 251.69
 26s] Train Epoch: 3 [7680/13374 (57%)]	Loss: 251.98
 28s] Tra

evaluating trained model ...

Test set: Accuracy: 5581/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Dutch
Nako is Japanese
 50s] Train Epoch: 21 [2560/13374 (19%)]	Loss: 41.88
 52s] Train Epoch: 21 [5120/13374 (38%)]	Loss: 42.59
 54s] Train Epoch: 21 [7680/13374 (57%)]	Loss: 43.75
 56s] Train Epoch: 21 [10240/13374 (77%)]	Loss: 45.43
 58s] Train Epoch: 21 [12800/13374 (96%)]	Loss: 44.89
evaluating trained model ...

Test set: Accuracy: 5602/6700 (84%)

Sung is Chinese
Jungwoo is Russian
Soojin is Dutch
Nako is Japanese
 2s] Train Epoch: 22 [2560/13374 (19%)]	Loss: 39.70
 3s] Train Epoch: 22 [5120/13374 (38%)]	Loss: 40.20
 5s] Train Epoch: 22 [7680/13374 (57%)]	Loss: 40.14
 8s] Train Epoch: 22 [10240/13374 (77%)]	Loss: 41.35
 10s] Train Epoch: 22 [12800/13374 (96%)]	Loss: 40.78
evaluating trained model ...

Test set: Accuracy: 5566/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Dutch
Nako is Japanese
 13s] Train Epoch: 23 [2560/13374 (19%)]	Loss: 39.94
 15s] Train 

 38s] Train Epoch: 40 [10240/13374 (77%)]	Loss: 15.54
 40s] Train Epoch: 40 [12800/13374 (96%)]	Loss: 15.83
evaluating trained model ...

Test set: Accuracy: 5588/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Polish
Nako is Japanese
 43s] Train Epoch: 41 [2560/13374 (19%)]	Loss: 13.44
 45s] Train Epoch: 41 [5120/13374 (38%)]	Loss: 14.45
 47s] Train Epoch: 41 [7680/13374 (57%)]	Loss: 14.80
 49s] Train Epoch: 41 [10240/13374 (77%)]	Loss: 14.96
 51s] Train Epoch: 41 [12800/13374 (96%)]	Loss: 14.94
evaluating trained model ...

Test set: Accuracy: 5561/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Polish
Nako is Japanese
 54s] Train Epoch: 42 [2560/13374 (19%)]	Loss: 11.74
 56s] Train Epoch: 42 [5120/13374 (38%)]	Loss: 12.64
 58s] Train Epoch: 42 [7680/13374 (57%)]	Loss: 13.14
 0s] Train Epoch: 42 [10240/13374 (77%)]	Loss: 14.05
 2s] Train Epoch: 42 [12800/13374 (96%)]	Loss: 13.93
evaluating trained model ...

Test set: Accuracy: 5562/6700 (83%)

Sung is Chinese
Jungw

 23s] Train Epoch: 60 [2560/13374 (19%)]	Loss: 20.63
 25s] Train Epoch: 60 [5120/13374 (38%)]	Loss: 18.08
 27s] Train Epoch: 60 [7680/13374 (57%)]	Loss: 17.98
 29s] Train Epoch: 60 [10240/13374 (77%)]	Loss: 18.28
 31s] Train Epoch: 60 [12800/13374 (96%)]	Loss: 19.09
evaluating trained model ...

Test set: Accuracy: 5533/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Polish
Nako is Japanese
 34s] Train Epoch: 61 [2560/13374 (19%)]	Loss: 16.93
 36s] Train Epoch: 61 [5120/13374 (38%)]	Loss: 18.63
 38s] Train Epoch: 61 [7680/13374 (57%)]	Loss: 18.06
 40s] Train Epoch: 61 [10240/13374 (77%)]	Loss: 18.64
 42s] Train Epoch: 61 [12800/13374 (96%)]	Loss: 18.54
evaluating trained model ...

Test set: Accuracy: 5507/6700 (82%)

Sung is Chinese
Jungwoo is Russian
Soojin is Irish
Nako is Japanese
 46s] Train Epoch: 62 [2560/13374 (19%)]	Loss: 19.57
 48s] Train Epoch: 62 [5120/13374 (38%)]	Loss: 20.10
 50s] Train Epoch: 62 [7680/13374 (57%)]	Loss: 19.62
 52s] Train Epoch: 62 [10240/13374 (

 4s] Train Epoch: 79 [12800/13374 (96%)]	Loss: 9.60
evaluating trained model ...

Test set: Accuracy: 5581/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Dutch
Nako is Japanese
 8s] Train Epoch: 80 [2560/13374 (19%)]	Loss: 7.86
 10s] Train Epoch: 80 [5120/13374 (38%)]	Loss: 7.62
 11s] Train Epoch: 80 [7680/13374 (57%)]	Loss: 9.50
 13s] Train Epoch: 80 [10240/13374 (77%)]	Loss: 9.81
 15s] Train Epoch: 80 [12800/13374 (96%)]	Loss: 9.89
evaluating trained model ...

Test set: Accuracy: 5554/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Dutch
Nako is Japanese
 19s] Train Epoch: 81 [2560/13374 (19%)]	Loss: 8.35
 21s] Train Epoch: 81 [5120/13374 (38%)]	Loss: 8.74
 23s] Train Epoch: 81 [7680/13374 (57%)]	Loss: 9.33
 25s] Train Epoch: 81 [10240/13374 (77%)]	Loss: 9.78
 27s] Train Epoch: 81 [12800/13374 (96%)]	Loss: 9.78
evaluating trained model ...

Test set: Accuracy: 5564/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Dutch
Nako is Japanese
 30s] Train Epoch: 8

 46s] Train Epoch: 99 [7680/13374 (57%)]	Loss: 29.48
 48s] Train Epoch: 99 [10240/13374 (77%)]	Loss: 28.24
 50s] Train Epoch: 99 [12800/13374 (96%)]	Loss: 26.79
evaluating trained model ...

Test set: Accuracy: 5515/6700 (82%)

Sung is Chinese
Jungwoo is Russian
Soojin is Polish
Nako is Japanese
 54s] Train Epoch: 100 [2560/13374 (19%)]	Loss: 15.94
 55s] Train Epoch: 100 [5120/13374 (38%)]	Loss: 15.27
 57s] Train Epoch: 100 [7680/13374 (57%)]	Loss: 14.91
 59s] Train Epoch: 100 [10240/13374 (77%)]	Loss: 14.98
 1s] Train Epoch: 100 [12800/13374 (96%)]	Loss: 14.81
evaluating trained model ...

Test set: Accuracy: 5547/6700 (83%)

Sung is Chinese
Jungwoo is Russian
Soojin is Polish
Nako is Japanese
